In [57]:
import pandas as pd
import numpy as np

In [ ]:
nonhouse = pd.read_excel("./EA_Data/pp_nonhouse.xlsx")

1. 인덱스 지정 및 수정
- 고성군1,2 구분
- 계 -> 서울특별시, ...

In [79]:
nonhouse = nonhouse[nonhouse["시군구"] != "세종시"]

In [ ]:
#고성군
nonhouse.loc[(nonhouse["구분"] == "강원") & (nonhouse["시군구"]=="고성군"), "시군구"] = "고성군1"
nonhouse.loc[(nonhouse["구분"] == "경남") & (nonhouse["시군구"]=="고성군"), "시군구"] = "고성군2"

In [97]:
do = { "강원": "강원도",
    "충북": "충청북도",
    "충남": "충청남도",
    "전북": "전라북도",
    "전남": "전라남도",
    "경북": "경상북도",
    "경남": "경상남도",
    "세종": "세종특별자치시",
    "제주": "제주특별자치도"}

nonhouse["시군구"] = nonhouse.apply(
    lambda row: do.get(row["구분"], row["시군구"]) if row["시군구"] == "계" else row["시군구"],
    axis=1
)
mapping = {
    "서울": "서울특별시",
    "부산": "부산광역시",
    "대구": "대구광역시",
    "인천": "인천광역시",
    "광주": "광주광역시",
    "대전": "대전광역시",
    "울산": "울산광역시",
}
# `구분` 열을 기준으로 `시군구` 값 매핑
nonhouse["시군구"] = nonhouse["구분"].map(mapping).fillna(nonhouse["시군구"])


In [63]:
result = pd.read_excel("ecm_result1.9.xlsx")

In [98]:
nonhouse.reset_index(inplace=True)

In [100]:
nonhouse.drop(columns=["level_0","index"])

,월,구분,시군구,미분양
0,2008-12,서울,서울특별시,"2,486"
1,2008-12,부산,부산광역시,"13,997"
2,2008-12,대구,대구광역시,"21,379"
3,2008-12,인천,인천광역시,"1,647"
4,2008-12,광주,광주광역시,"12,384"
...,...,...,...,...
2577,2022-12,경남,거창군,0
2578,2022-12,경남,합천군,0
2579,2022-12,제주,제주특별자치도,"1,676"
2580,2022-12,제주,제주시,863


연도별 개수 체크

In [101]:
before = 2008
year_gap =[0]

for i in range(len(nonhouse)):
    if int(nonhouse.loc[i,"월"][:4]) != before:
        year_gap.append(nonhouse.iloc[i,:].name)
        before += 1
        
year_gap.append(len(nonhouse)-1)
#year_gap

In [102]:
print(year_gap)
print(len(year_gap))


[0, 174, 348, 522, 694, 867, 1040, 1212, 1384, 1556, 1727, 1898, 2069, 2240, 2411, 2581]
16


인덱스 지정

In [103]:
custom_order = list(result.loc[:178,"시군구"])

In [104]:
duplicates = [x for x in set(custom_order) if custom_order.count(x) > 1]
print("중복값:", duplicates) 
    

중복값: []


1. 5개 추가: "통합창원시","통합청주시",("여주시","여주군"),("당진시","당진군"),"세종특별자치시"
2. 180개씩 잘라서 reindex
3. concat으로 합치기

In [105]:
result_form = pd.DataFrame(columns=["월", "구분", "미분양"]) #새로 담을 데이터 프레임

In [ ]:
for i, year in zip(range(len(year_gap)), range(2008,2023)):
        
    subsample = nonhouse[year_gap[i]:year_gap[i+1]]

    new_rows = pd.DataFrame([
        {"미분양": 0, "월": year, "구분": "경남", "시군구": "통합창원시"},
        {"미분양": 0, "월": year, "구분": "충남", "시군구": "통합청주시"},
    ])
    if "세종특별자치시" not in subsample["시군구"].to_list():
        print(0)
        new_rows = pd.concat([
            new_rows, pd.DataFrame([{"미분양": 0, "월": year, "구분": "세종", "시군구": "세종특별자치시"}])
        ])
    if "마산시" not in subsample["시군구"].to_list():
        print(1)
        new_rows = pd.concat([
            new_rows, pd.DataFrame([{"미분양": 0, "월": year, "구분": "경남", "시군구": "마산시"}])
        ])
    if "진해시" not in subsample["시군구"].to_list():
        print(2)
        new_rows = pd.concat([
            new_rows, pd.DataFrame([{"미분양": 0, "월": year, "구분": "경남", "시군구": "진해시"}])
        ])
    if "연기군" not in subsample["시군구"].to_list():
        print(3)
        new_rows = pd.concat([
            new_rows, pd.DataFrame([{"미분양": 0, "월": year, "구분": "충남", "시군구": "연기군"}])
        ])
    if "청원군" not in subsample["시군구"].to_list():
        print(4)
        new_rows = pd.concat([
            new_rows, pd.DataFrame([{"미분양": 0, "월": year, "구분": "충북", "시군구": "청원군"}])
        ])
    if "여주군" not in subsample["시군구"].to_list():
        print(5)
        new_rows = pd.concat([
            new_rows, pd.DataFrame([{"미분양": 0, "월": year, "구분": "경기", "시군구": "여주군"}])
        ])
    if "여주시" not in subsample["시군구"].to_list():
        print(6)
        new_rows = pd.concat([
            new_rows, pd.DataFrame([{"미분양": 0, "월": year, "구분": "경기", "시군구": "여주시"}])
        ])
    if "당진군" not in subsample["시군구"].to_list():
        print(7)
        new_rows = pd.concat([
            new_rows, pd.DataFrame([{"미분양": 0, "월": year, "구분": "충남", "시군구": "당진군"}])
        ])
    if "당진시" not in subsample["시군구"].to_list():
        print(8)
        new_rows = pd.concat([
            new_rows, pd.DataFrame([{"미분양": 0, "월": year, "구분": "충남", "시군구": "당진시"}])
        ])

    # 데이터 병합
    subsample = pd.concat([subsample, new_rows])
    subsample_index = subsample.set_index(keys="시군구")
    print(len(subsample_index),len(custom_order))
    print(subsample_index.tail(3))
    subsample_sorted = subsample_index.reindex(custom_order)
    result_form = pd.concat([result_form, subsample_sorted])

In [109]:
result_form.to_excel("nonhouse2.0.xlsx")